In [ ]:
pip install langchain-openai langchain-exa

In [ ]:
pip install --upgrade openai langchain

In [ ]:
import os
import pandas as pd
import numpy as np
import openpyxl

from openpyxl import load_workbook
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from IPython.display import Markdown
from exa_py import Exa
from langchain_core.tools import tool
from openai import OpenAI
from tqdm.notebook import tqdm
from statistics import mode

load_dotenv()

file_path = "Investments.xlsx"

exa = Exa(api_key=os.environ["EXA_API_KEY"])

# we will ask exa to search for relevant documents 5 times: each time it will output 10 documents 
number_of_exa_search_repetitions = 2
# we will prompt the generative LLM 10 times for each exa search --> this results in prompting the llm 10 times with a given set of documents, 
# then another 10 time swith a (potentially) different set, etc... (done 5 times) --> results in 50 predictions per year per technology   
number_of_llm_prompting_repetitions = 10
technologies = ["AI"]
years = range(2015,2035)

LLM_MODEL = "gpt-4o"
LLM_TEMPERATURE = 0.5
base_model = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model=LLM_MODEL, temperature=LLM_TEMPERATURE)

AI_AGENT_PROMPT = """You are an expert financial assistant. You use xml-formatted context to answer people's questions. You always provide your sources.
        The client will ask you an investment question. Use the Exa search tool to look for historical documents regarding investments in the scope of 
        the client's request and that are most helpful to answer that request. 
        Finally, using the selected documents as a guide/help, provide a clear and succint answer to the client's request. Do not call the tool again;
        just answer using the context provided and if no useful context is available, respond using your own knowledge. The documents might not 
        contain the exact answer but you still need to answer the request as the client expects."""

@tool
def search_and_contents(query: str, start_published_date: str, end_published_date: str):
    """Search for webpages based on the query and retrieve their contents."""
    # This combines two API endpoints: search and contents retrieval
    params = {
        "query": query,
        "use_autoprompt": True,
        "start_published_date": start_published_date,
        "end_published_date": end_published_date,
        "num_results":10,
        "text": True, 
        "highlights": True}
    return exa.search_and_contents(**params)
        
for technology in tqdm(technologies): 
    first_guess_investment_predictions = []
    mean_investment_predictions = []
    median_investment_predictions = []
    mode_investment_predictions = [] 
    for year in tqdm(years):
            
        tools = [search_and_contents]
        tools_by_name = {tool.name: tool for tool in tools}
        
        exa_model = base_model.bind_tools(tools)

        user_query = f"Predict the total global investments (not market size) to the field of {technology} in the year {year}. If not sure, speculate. Provide one number as your prediction. In your research, restrict your information sources to those published from 2010-01-01 to {year-1}-12-31, inclusive."
            
        task_str = f"User query: {user_query}"
            
        investment_predictions = []
        investment_predictions_content = []
        for _ in tqdm(range(number_of_exa_search_repetitions)):            
            messages = [
                SystemMessage(AI_AGENT_PROMPT),
                HumanMessage(task_str),
            ]
            # Invoke the financial assistant model with the messages
            exa_model_output = exa_model.invoke(messages)
            
            document_prompt = """
            <source>
                <url>{url}</url>
                <highlights>{highlights}</highlights>
            </source>
            """
            
            # If exa tool model made tool calls, invoke the tool
            if exa_model_output.tool_calls:
                response_list = []
                id_list = []
                for tool_call in exa_model_output.tool_calls:
                    tool = tools_by_name[tool_call["name"]]
                    tool_response = tool.invoke(tool_call["args"])
            
                    response_list.append(tool_response)
                    id_list.append(tool_call["id"])

                if(year<=2025):
                    prohibited_dates = list(range(year, 2026))
                    formatted_documents_with_ids = [(id_, "\n".join([document_prompt.format(url=result.url , highlights=result.highlights) for result in response.results if result.published_date and 
                                       all([str(date) not in result.title for date in prohibited_dates]) and 
                                       all([str(date) not in result.url for date in prohibited_dates]) and
                                       all([str(date) not in result.text for date in prohibited_dates])])) for id_, response in zip(id_list, response_list)]
                else:
                    formatted_documents_with_ids = [(id_, "\n".join([document_prompt.format(url=result.url , highlights=result.highlights) for result in response.results])) for id_, response in zip(id_list, response_list)]
                
                for i in range(len(formatted_documents_with_ids)):
                    if(formatted_documents_with_ids[i][1] == ""):
                       formatted_documents_with_ids[i] = (formatted_documents_with_ids[i][0], "No relevant documents were found for this query.")
                # Message list with the retrieved documents for the base model
                # print(len(formatted_documents_with_ids))
                # print(exa_model_output)
                # print(id_list)
                # print(exa_model_output.tool_calls)
                messages = [
                    SystemMessage(AI_AGENT_PROMPT),
                    HumanMessage(task_str),
                    exa_model_output,
                    *[
                        ToolMessage(doc, tool_call_id=tool_call_id)
                        for (tool_call_id, doc) in formatted_documents_with_ids
                    ],
                ]

                for _ in tqdm(range(number_of_llm_prompting_repetitions)):
                    # Invoke the base model with the messages
                    exa_model_output_final = exa_model.invoke(messages)

                    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
                    
                    response = client.responses.create(
                        model="o4-mini",
                        input="Extract the mentioned final predicted investment value in this text as a number (omit the monetary signs and units). Example: if it's said: $20 billion, you should output 20000000000. If a range is given take the middle point of the range. If no prediction is given, output -1.\nText: " + exa_model_output_final.content)
                    
                    investment_predictions.append(response.output_text) 
                    investment_predictions_content.append(exa_model_output_final.content)

            else:
                # if the exa-tool-enhanced llm (i.e. AI agent) decided not to call the tool (i.e exa in this case), we log this before going to the next iteration
                investment_predictions.append("-1") 
                investment_predictions_content.append("no tool calls")
        
        cleaned_investment_predictions = [int(investment_prediction) for investment_prediction in investment_predictions if(investment_prediction != "-1" and investment_prediction != "no tool calls")]
        
        if(cleaned_investment_predictions):
            first_guess_investment_predictions.append(cleaned_investment_predictions[0])
            mean_investment_predictions.append(np.mean(cleaned_investment_predictions))
            median_investment_predictions.append(np.median(cleaned_investment_predictions))
            mode_investment_predictions.append(mode(cleaned_investment_predictions))
        else:
            first_guess_investment_predictions.append(-1)
            mean_investment_predictions.append(-1)
            median_investment_predictions.append(-1)
            mode_investment_predictions.append(-1)

    df_tech = pd.read_excel(file_path, sheet_name=technology)
    df_tech["agentic RAG (first guess)"] = first_guess_investment_predictions
    df_tech["agentic RAG (mean)"] = mean_investment_predictions
    df_tech["agentic RAG (median)"] = median_investment_predictions
    df_tech["agentic RAG (mode)"] = mode_investment_predictions
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:        
        # Write updated dataframe to the correct sheet   
        df_tech.to_excel(writer, sheet_name=technology, index=False)

In [ ]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from sklearn.linear_model import LinearRegression

file_path = "Investments.xlsx"

# Load existing workbook
book = load_workbook(file_path)
# get the names of all sheets in the file
sheet_names = [worksheet.title for worksheet in book.worksheets]

df = pd.read_excel("ground truth investments.xlsx")

for sheet_name in sheet_names:
    predictions = []
    for i in range(5,15):
        X = df.iloc[:i][["Year"]]
        y = df.iloc[:i][[sheet_name]]
        reg = LinearRegression().fit(X, y)
        predictions = predictions + reg.predict(df.iloc[i:i+1][["Year"]]).flatten().tolist()
        
    X = df[["Year"]]
    y = df[[sheet_name]]
    reg = LinearRegression().fit(X, y)
    predictions = predictions + reg.predict(np.array(range(2025, 2035)).reshape(-1,1)).flatten().tolist()

    # read just one sheet to dataframe:
    df_tech = pd.read_excel(file_path, sheet_name=sheet_name)
    df_tech["linear regression"] = predictions
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:        
        # Write updated dataframe to the correct sheet   
        df_tech.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib import colormaps
import openpyxl
from openpyxl import load_workbook
import pandas as pd

file_path = "Investments.xlsx"

# Load existing workbook
book = load_workbook(file_path)
# get the names of all sheets in the file
sheet_names = [worksheet.title for worksheet in book.worksheets]

sheet_name = sheet_names[6]
df = pd.read_excel(file_path, sheet_name=sheet_name)

def custom_y_axis_formatter(x, i):
    if((x/1e9)<1):
        return f'${x/1e6:.0f}M'
    elif((x/1e9)>=1 and (x/1e9)<1000):
        return f'${x/1e9:.0f}B'
    else:
        return f'${x/1e12:.0f}T'

cmap = colormaps['tab10']
colors = cmap.colors[:6]

# Create the plot
fig, ax = plt.subplots(figsize=(10, 6))
df["Year"] = df["Year"].astype(str)
# Plot years 2015–2024 (i.e., ground truth and predictions)
sns.lineplot(data=df.iloc[:10], x="Year", y="ground truth", ax=ax, color=colors[2], label="Ground Truth", marker="o")
sns.lineplot(data=df.iloc[:10], x="Year", y="agentic RAG (first guess)", ax=ax, color=colors[0], label="Agentic RAG (first guess)", marker="o")
sns.lineplot(data=df.iloc[:10], x="Year", y="agentic RAG (mean)", ax=ax, color=colors[1], label="Agentic RAG (mean)", marker="o")
sns.lineplot(data=df.iloc[:10], x="Year", y="agentic RAG (median)", ax=ax, color=colors[3], label="Agentic RAG (median)", marker="o")
sns.lineplot(data=df.iloc[:10], x="Year", y="agentic RAG (mode)", ax=ax, color=colors[4], label="Agentic RAG (mode)", marker="o")
sns.lineplot(data=df.iloc[:10], x="Year", y="linear regression", ax=ax, color=colors[5], label="Linear Regression", marker="o")

# Plot years 2025–2034 (projections only)
sns.lineplot(data=df.iloc[9:], x="Year", y="agentic RAG (first guess)", ax=ax, color=colors[0], linestyle='--', marker="o")
sns.lineplot(data=df.iloc[9:], x="Year", y="agentic RAG (mean)", ax=ax, color=colors[1], linestyle='--', marker="o")
sns.lineplot(data=df.iloc[9:], x="Year", y="agentic RAG (median)", ax=ax, color=colors[3], linestyle='--', marker="o")
sns.lineplot(data=df.iloc[9:], x="Year", y="agentic RAG (mode)", ax=ax, color=colors[4], linestyle='--', marker="o")
sns.lineplot(data=df.iloc[9:], x="Year", y="linear regression", ax=ax, color=colors[5], linestyle='--', marker="o")

ax.set_yscale("log") 
ax.set_yticks([0.5e8, 1e8, 1e9, 1e10, 1e11, 1e12, 1e13, 0.2e14])
# Format y-axis as currency
ax.yaxis.set_major_formatter(mtick.FuncFormatter(custom_y_axis_formatter))

# Label and title
# ax.set_title("Agentic RAG - " + sheet_name + " Investment - Ground Truth vs Forecasts (log scale)")
ax.set_ylabel("Total Global Investment (log scale)")
ax.set_xlabel("Year")

# Hide the first y-tick label
ax.get_yticklabels()[0].set_visible(False)
ax.get_yticklabels()[-1].set_visible(False)

# Legend
ax.legend(title="Method", loc="upper left")

plt.tight_layout()
plt.savefig(sheet_name + " investments ground truth vs forecasts log scale.png")
plt.show()

In [ ]:
import pandas as pd

def mnae(x,y):
    """x: ground truth
       y: prediction"""
    return np.mean(np.abs(x-y)/x)

def mnse(x,y):
    """x: ground truth
       y: prediction"""
    return np.mean((x-y)**2/x)

sheet_name = "Electronics & Photonics"
file_path = "AI investments draft agentic rag investments.xlsx"
df = pd.read_excel(file_path, sheet_name=sheet_name)

mae_first_guess = mnse(df.iloc[:10]["ground truth"], df.iloc[:10]["agentic RAG (first guess)"])
mae_mean = mnse(df.iloc[:10]["ground truth"], df.iloc[:10]["agentic RAG (mean)"])
mae_median = mnse(df.iloc[:10]["ground truth"], df.iloc[:10]["agentic RAG (median)"])
mae_mode = mnse(df.iloc[:10]["ground truth"], df.iloc[:10]["agentic RAG (mode)"])
mae_linear_regression = mnse(df.iloc[:10]["ground truth"], df.iloc[:10]["linear regression"])

mae = [round(mae_first_guess, 2), round(mae_mean, 2), round(mae_median, 2), round(mae_mode, 2), round(mae_linear_regression, 2)]
print(min(mae))
print(mae)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

sheet_name = "Robotics"
file_path = "Investments.xlsx"
df = pd.read_excel(file_path, sheet_name=sheet_name)

fig, ax = plt.subplots()
plot_years = np.arange(2015, 2025)
ax.plot(plot_years, np.abs(df.iloc[:10]["ground truth"] - df.iloc[:10]["agentic RAG (first guess)"])/df.iloc[:10]["ground truth"], color=colors[0], marker='o', label="Agentic RAG (first guess)")
ax.plot(plot_years, np.abs(df.iloc[:10]["ground truth"] - df.iloc[:10]["agentic RAG (mean)"])/df.iloc[:10]["ground truth"], color=colors[1], marker='o', label="Agentic RAG (mean)")
ax.plot(plot_years, np.abs(df.iloc[:10]["ground truth"] - df.iloc[:10]["agentic RAG (median)"])/df.iloc[:10]["ground truth"], color=colors[3], marker='o', label="Agentic RAG (median)")
ax.plot(plot_years, np.abs(df.iloc[:10]["ground truth"] - df.iloc[:10]["agentic RAG (mode)"])/df.iloc[:10]["ground truth"], color=colors[4], marker='o', label="Agentic RAG (mode)")

# Create inset axes (zoomed-in version)
axins = inset_axes(ax, width="60%", height="45%", loc='lower center', borderpad=5)

# Zoomed-in plot for 2015–2024
axins.plot(plot_years, np.abs(df.iloc[:10]["ground truth"] - df.iloc[:10]["agentic RAG (first guess)"])/df.iloc[:10]["ground truth"], color=colors[0], marker='o', label="Agentic RAG (first guess)")
axins.plot(plot_years, np.abs(df.iloc[:10]["ground truth"] - df.iloc[:10]["agentic RAG (mean)"])/df.iloc[:10]["ground truth"], color=colors[1], marker='o', label="Agentic RAG (mean)")
axins.plot(plot_years, np.abs(df.iloc[:10]["ground truth"] - df.iloc[:10]["agentic RAG (median)"])/df.iloc[:10]["ground truth"], color=colors[3], marker='o', label="Agentic RAG (median)")
axins.plot(plot_years, np.abs(df.iloc[:10]["ground truth"] - df.iloc[:10]["agentic RAG (mode)"])/df.iloc[:10]["ground truth"], color=colors[4], marker='o', label="Agentic RAG (mode)")

axins.set_title("Zoom on investments", fontsize=10)
axins.set_xticks(plot_years)
axins.set_xticklabels(plot_years)
axins.tick_params(axis='x', rotation=45)
axins.set_ylim(0,5)

ax.set_xticks(plot_years)
ax.set_xlabel('Year')
ax.set_ylabel('Normalized Absolute Error')
ax.legend(title='Method', loc='upper left')
ax.set_ylim((0,40))

plt.tight_layout()
plt.savefig("nae_"+sheet_name+".png")
plt.show()